In [71]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [72]:
import pandas as pd
import numpy as np
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

Mounting drive for file access in drive

In [73]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [74]:
df = spark.read.option('multiline', True).json('/content/drive/My Drive/Colab Notebooks/donation_np.json')

Column Rename

In [75]:
df=df.withColumnRenamed("Contribution Mode","mode_of_payment")
df=df.withColumnRenamed("Financial Year","fin_year")
df=df.withColumnRenamed("PAN Given","pan_given")

Encryption of Address

In [76]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
import hashlib
def encrypt_value(address):
    sha_value = hashlib.sha256(address.encode()).hexdigest()
    return sha_value

spark_udf = udf(encrypt_value, StringType())
#data = db.withColumn('encrypted_value',spark_udf('Address'))
df=df.withColumn('Address',md5(col("Address")))

Categorization of Mode of payment

In [77]:
payment_type_list = list(df.rdd.map(lambda x:x.mode_of_payment).collect())

In [78]:
len(payment_type_list)


13573

In [79]:
categorized_list=[]
#Try Except is to handle null values
for mode in payment_type_list:
  try:
    if "cheque" in mode.lower():
      categorized_list.append('Cheque')
    elif "bank" in mode.lower():
      categorized_list.append('Bank')
    elif "cash" in mode.lower():
      categorized_list.append('Cash')
    else:
      categorized_list.append('Others')
  except:
      categorized_list.append('Others')
  


In [80]:
categorized_list

['Cash',
 'Bank',
 'Cheque',
 'Bank',
 'Bank',
 'Others',
 'Others',
 'Bank',
 'Others',
 'Bank',
 'Bank',
 'Others',
 'Cheque',
 'Cheque',
 'Cheque',
 'Bank',
 'Bank',
 'Bank',
 'Others',
 'Bank',
 'Bank',
 'Bank',
 'Cheque',
 'Bank',
 'Bank',
 'Bank',
 'Others',
 'Cash',
 'Bank',
 'Bank',
 'Bank',
 'Bank',
 'Bank',
 'Bank',
 'Bank',
 'Bank',
 'Bank',
 'Others',
 'Bank',
 'Bank',
 'Others',
 'Others',
 'Bank',
 'Bank',
 'Others',
 'Cash',
 'Cash',
 'Cash',
 'Cash',
 'Cash',
 'Cash',
 'Cash',
 'Cash',
 'Cash',
 'Cash',
 'Cheque',
 'Cash',
 'Others',
 'Others',
 'Cash',
 'Cash',
 'Cheque',
 'Cash',
 'Cash',
 'Cash',
 'Cheque',
 'Cash',
 'Cash',
 'Cash',
 'Cash',
 'Cash',
 'Cash',
 'Cash',
 'Cash',
 'Cash',
 'Cash',
 'Cash',
 'Cheque',
 'Cash',
 'Cash',
 'Cheque',
 'Cheque',
 'Cash',
 'Bank',
 'Others',
 'Cash',
 'Cheque',
 'Cheque',
 'Cheque',
 'Bank',
 'Others',
 'Others',
 'Bank',
 'Bank',
 'Bank',
 'Others',
 'Others',
 'Bank',
 'Bank',
 'Bank',
 'Cheque',
 'Cheque',
 'Others',
 'Cas

In [81]:
new_df = spark.createDataFrame([(element,) for element in categorized_list], ['mode_of_payment'])

In [82]:
df=df.drop(col('mode_of_payment'))

In [83]:
#df = df.withColumn("mode_of_payment",col(new_list))

In [84]:
from pyspark.sql.functions import col

In [85]:
# Used Horizontal Dataframe merging using join operation
from pyspark.sql.functions import monotonically_increasing_id
df = df.withColumn("id",monotonically_increasing_id() )
df.show()
new_df = new_df.withColumn( "id", monotonically_increasing_id() )
new_df.show()
horiztnlcombined_data = df.join(new_df,df.id == new_df.id, how='inner').drop(df.id)
horiztnlcombined_data.show()


+--------------------+--------+--------+--------------------+---------+------+------+-------+-------+-------+-------+-------+------+---+
|             Address|  Amount|fin_year|                Name|pan_given| Party|  Type|field10|field11|field12|field13|field14|field9| id|
+--------------------+--------+--------+--------------------+---------+------+------+-------+-------+-------+-------+-------+------+---+
|44bca5383d6dae229...| 3000000| 2010-11|          Aziz Pasha|        Y|   CPI|Others|       |       |       |       |       |      |  0|
|3d5fc135c2defdd47...|10000000| 2014-15|    V K Ramachandran|        Y|CPI(M)|Others|       |       |       |       |       |      |  1|
|425cb8edfacd38d97...|  108000| 2014-15|  Dr. Manmohan Singh|        N|   INC|Others|       |       |       |       |       |      |  2|
|39532b270ae265a0e...|   54000| 2011-12| Dr. Manda Jagnathan|        N|   INC|Others|       |       |       |       |       |      |  3|
|e5eaf382bb92f67dd...|   54000| 2011-12| 

In [86]:
df  = horiztnlcombined_data

In [87]:
df.show()

+--------------------+--------+--------+--------------------+---------+------+------+-------+-------+-------+-------+-------+------+---------------+---+
|             Address|  Amount|fin_year|                Name|pan_given| Party|  Type|field10|field11|field12|field13|field14|field9|mode_of_payment| id|
+--------------------+--------+--------+--------------------+---------+------+------+-------+-------+-------+-------+-------+------+---------------+---+
|44bca5383d6dae229...| 3000000| 2010-11|          Aziz Pasha|        Y|   CPI|Others|       |       |       |       |       |      |           Cash|  0|
|3d5fc135c2defdd47...|10000000| 2014-15|    V K Ramachandran|        Y|CPI(M)|Others|       |       |       |       |       |      |           Bank|  1|
|425cb8edfacd38d97...|  108000| 2014-15|  Dr. Manmohan Singh|        N|   INC|Others|       |       |       |       |       |      |         Cheque|  2|
|39532b270ae265a0e...|   54000| 2011-12| Dr. Manda Jagnathan|        N|   INC|Othe

Sum ,Average and Count Columns

In [88]:
from collections import Counter

In [89]:
party_dict = dict(Counter(list(df.rdd.map(lambda x:x.Party).collect())))

In [90]:
party_dict

{'BJP': 4734, 'CPI': 278, 'CPI(M)': 330, 'INC': 1772, 'NCP': 54}

In [91]:
INC_SUM_LTD=0
BJP_SUM_LTD=0
NCP_SUM_LTD=0
CPI_SUM_LTD=0
CPIM_SUM_LTD=0

INC_mx=0
BJP_mx=0
NCP_mx=0
CPI_mx=0
CPIM_mx=0

for party in df.select('Party',"Amount").collect():
  if party[0] == "CPI":
    CPI_SUM_LTD+=int(party[1])
    CPI_mx= CPI_mx if CPI_mx > int(party[1]) else int(party[1])
  elif party[0] ==  "NCP":
    NCP_SUM_LTD+=int(party[1])
    NCP_mx= NCP_mx if NCP_mx > int(party[1]) else int(party[1])
  elif party[0] == "BJP":
    BJP_SUM_LTD+=int(party[1])
    BJP_mx= BJP_mx if BJP_mx > int(party[1]) else int(party[1])
  elif party[0] == "INC":
    INC_SUM_LTD+=int(party[1])
    INC_mx= INC_mx if INC_mx > int(party[1]) else int(party[1])
  else:
    CPIM_SUM_LTD+=int(party[1])
    CPIM_mx = CPIM_mx if CPIM_mx > int(party[1]) else int(party[1])


In [92]:
df = df.withColumn("INC_SUM_LTD",when((df.Party == "INC"),lit(INC_SUM_LTD)).otherwise(lit(0)))
df = df.withColumn("BJP_SUM_LTD",when((df.Party == "BJP"),lit(BJP_SUM_LTD)).otherwise(lit(0)))
df = df.withColumn("NCP_SUM_LTD",when((df.Party == "NCP"),lit(NCP_SUM_LTD)).otherwise(lit(0)))
df = df.withColumn("CPI_SUM_LTD",when((df.Party == "CPI"),lit(CPI_SUM_LTD)).otherwise(lit(0)))
df = df.withColumn("CPIM_SUM_LTD",when((df.Party == "CPI(M)"),lit(CPIM_SUM_LTD)).otherwise(lit(0)))

In [93]:
df = df.withColumn("INC_COUNT_LTD",when((df.Party == "INC"),lit(party_dict['INC'])).otherwise(lit(0)))
df = df.withColumn("BJP_COUNT_LTD",when((df.Party == "BJP"),lit(party_dict['BJP'])).otherwise(lit(0)))
df = df.withColumn("NCP_COUNT_LTD",when((df.Party == "NCP"),lit(party_dict['NCP'])).otherwise(lit(0)))
df = df.withColumn("CPI_COUNT_LTD",when((df.Party == "CPI"),lit(party_dict['CPI'])).otherwise(lit(0)))
df = df.withColumn("CPIM_COUNT_LTD",when((df.Party == "CPI(M)"),lit(party_dict['CPI(M)'])).otherwise(lit(0)))

In [94]:
df = df.withColumn("INC_AVG_LTD",when((df.Party == "INC"),lit(INC_SUM_LTD/party_dict['INC'])).otherwise(lit(0)))
df = df.withColumn("BJP_AVG_LTD",when((df.Party == "BJP"),lit(BJP_SUM_LTD/party_dict['BJP'])).otherwise(lit(0)))
df = df.withColumn("NCP_AVG_LTD",when((df.Party == "NCP"),lit(NCP_SUM_LTD/party_dict['NCP'])).otherwise(lit(0)))
df = df.withColumn("CPI_AVG_LTD",when((df.Party == "CPI"),lit(CPI_SUM_LTD/party_dict['CPI'])).otherwise(lit(0)))
df = df.withColumn("CPIM_AVG_LTD",when((df.Party == "CPI(M)"),lit(CPIM_SUM_LTD/party_dict['CPI(M)'])).otherwise(lit(0)))

Columns by fin year and by party


In [95]:
from pyspark.sql.types import IntegerType
df = df.withColumn("Amount",col("Amount").cast(IntegerType()))
new = df.groupBy('fin_year','Party').sum("Amount")

In [96]:
for i,j,k in new.select('fin_year','Party',"sum(Amount)").collect():
  df = df.withColumn("{}_{}_SUM".format(i,j),when( (df.Party == "{}".format(j)) & (df.fin_year == "{}".format(i)) ,lit(k) ).otherwise(lit(0)))

In [97]:
df.show()

+--------------------+--------+--------+--------------------+---------+------+------+-------+-------+-------+-------+-------+------+---------------+---+-----------+-----------+-----------+-----------+------------+-------------+-------------+-------------+-------------+--------------+------------------+----------------+-----------+------------------+------------------+---------------+---------------+---------------+---------------+------------------+---------------+---------------+---------------+------------------+------------------+------------------+---------------+------------------+---------------+---------------+---------------+---------------+------------------+---------------+------------------+------------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+------------------+---------------+---------------+---------------+---------------+---------------+------------------+---------------+--------

In [98]:
mode_count_dict = dict(Counter(categorized_list))

In [99]:
mode_count_dict

{'Bank': 7790, 'Cash': 915, 'Cheque': 2081, 'Others': 2787}

Mode of payment Columns

In [100]:
df = df.withColumn("Bank_COUNT_LTD",when((df.mode_of_payment == "Bank"),lit(mode_count_dict['Bank'])).otherwise(lit(0)))
df = df.withColumn("Cheque_COUNT_LTD",when((df.mode_of_payment == "Cheque"),lit(mode_count_dict['Cheque'])).otherwise(lit(0)))
df = df.withColumn("Cash_COUNT_LTD",when((df.mode_of_payment == "Cash"),lit(mode_count_dict['Cash'])).otherwise(lit(0)))
df = df.withColumn("Others_COUNT_LTD",when((df.mode_of_payment == "Others"),lit(mode_count_dict['Others'])).otherwise(lit(0)))

Parquet File Format

In [103]:
df.write.parquet('Parquet_Dataframe.parquet')